__import des librairies nécessaires__

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm
import copy
import argparse
import os
import sys
#from utils import *

__import des autres fichiers__

In [2]:
print(os.getcwd())
os.chdir('..')

c:\Users\dmgtr\chocoEA\impl


In [3]:
print(os.getcwd())

c:\Users\dmgtr\chocoEA


In [4]:
from nv_orga.Models import Net, Net_eNTK, ResNet18, ResNet14
from nv_orga.FedAvg import average_models,client_update
from nv_orga.Eval import evaluate_many_models
from nv_orga.NTK import client_compute_eNTK
from nv_orga.NTK import compute_eNTK
from nv_orga.Scaffold import scaffold_update

__définition des hyperparamètres__

In [5]:
args = {
    'num_client': 5,
    'seed': 1234,
    'num_samples_per_client': 5000, #500 de base
    'rounds_stage1': 100, #100 de base
    'local_epochs_stage1': 5,
    'mini_batchsize_stage1': 64,
    #'local_lr_stage1': 0.0001,
    'local_lr_stage1': 0.1,
    'rounds_stage2': 30, #100 de base
    'local_steps_stage2': 100,
    'local_lr_stage2': 0.001,
    #'weight_decay' : 1e-4,
    #'momentum' : 0.9,
    }

__création d'un dossier de sauvegarde pour les modèles successifs du stage 1__

In [6]:
isExist = os.path.exists('data/ckpt_stage1')
if not isExist:
   os.makedirs('data/ckpt_stage1')

## __Stage 1__

### __hyperparamètres__

In [7]:
num_clients = args["num_client"]
num_rounds_stage1 = args["rounds_stage1"]
epochs_stage1 = args["local_epochs_stage1"]
batch_size_stage1 = args["mini_batchsize_stage1"]
lr_stage1 = args["local_lr_stage1"]
#weight_decay = args["weight_decay"]
#momentum = args["momentum"]

### __création des datasets décentralisés (ie non idd)__

In [8]:
## load les data MNIST, les transformer en tensor et les normaliser
traindata = datasets.MNIST('data/data_mnist', train=True, download=True,
                            transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.1307,), (0.3081,))]))

## load CIFAR-10 data, transform them into tensors, and normalize
traindata = datasets.CIFAR10('data/data_cifar10', train=True, download=True,
                            transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))

targets = torch.tensor(traindata.targets)
#target_labels = torch.stack([traindata.targets == i for i in range(10)], dim=0).int() # 10 x 60000 (one-hot qui détermine la label correpondant à la ligne)
target_labels = torch.stack([targets == i for i in range(10)], dim=0).int()


Files already downloaded and verified


In [9]:
target_labels_split = []
torch.manual_seed(args["seed"]) # pour que les splits soient les mêmes à chaque fois
torch.cuda.manual_seed(args["seed"])  # pour que les splits soient les mêmes à chaque fois

for i in range(num_clients):
    #index_split = torch.where(target_labels[(2 * i):(2 * (i + 1))].sum(0))[0] # on prend les labels 2i et 2i+1
    index_split = torch.where(target_labels[0:10].sum(0))[0] # on prend les labels 2i et 2i+1
    perm_split = torch.randperm(index_split.size(0)) # on mélange les indices
    index_split_subsample = index_split[perm_split[:args["num_samples_per_client"]]] 
    target_labels_split += [index_split_subsample] # on ajoute à la liste des labels splités

#Chacun des 5 clients reçoit num_samples_per_client images d'un des deux labels associés

In [10]:
# Training datasets (subsampled)
traindata_split = [torch.utils.data.Subset(traindata, tl) for tl in target_labels_split] # chaque élément contient les images et labels d'un client
train_loader = [torch.utils.data.DataLoader(train_subset, batch_size=batch_size_stage1, shuffle=True)
                for train_subset in traindata_split] # on crée les dataloader associés

### __création du dataset global de test__

In [11]:
# Test dataset (subsampled)
#testdata = datasets.MNIST('data/data_mnist', train=False,
#                          transform=transforms.Compose([transforms.ToTensor(),
#                                                        transforms.Normalize((0.1307,), (0.3081,))])) # on charge les données de test


#torch.manual_seed(args["seed"])
#torch.cuda.manual_seed(args["seed"])
#perm_split_test = torch.randperm(testdata.targets.shape[0])
#testdata_subset = torch.utils.data.Subset(testdata, perm_split_test[:1000])
#test_loader = torch.utils.data.DataLoader(testdata_subset, batch_size=batch_size_stage1, shuffle=False) #pas de shuffle pour le test

In [12]:
testdata = datasets.CIFAR10('data/data_cifar10', train=False,
                            transform=transforms.Compose([transforms.ToTensor(),
                                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))


# Convertir les cibles en tenseur si nécessaire
targets_cifar = torch.tensor(testdata.targets) if isinstance(testdata.targets, list) else testdata.targets

torch.manual_seed(args["seed"])
torch.cuda.manual_seed(args["seed"])

# Création d'un sous-ensemble aléatoire pour CIFAR10
perm_split_test = torch.randperm(targets_cifar.shape[0])
testdata_subset = torch.utils.data.Subset(testdata, perm_split_test[:10000]) # on prend 1000 images
test_loader = torch.utils.data.DataLoader(testdata_subset, batch_size=batch_size_stage1, shuffle=False)

### __modèle de réseau de neurones de base__

In [13]:
#global_model = Net() #modifié depuis Net().cuda() #Modèle fédéré
#client_models = [Net() for _ in range(num_clients)] #modifié depuis Net().cuda() #Modèles des clients
#for model in client_models:
    #model.load_state_dict(global_model.state_dict())
#opt = [optim.SGD(model.parameters(), lr=lr_stage1) for model in client_models]

In [14]:
global_model = ResNet18() #modifié depuis Net().cuda() #Modèle fédéré
client_models = [ResNet18() for _ in range(num_clients)] #modifié depuis Net().cuda() #Modèles des clients
for model in client_models:
    model.load_state_dict(global_model.state_dict())

opt = [optim.SGD(model.parameters(), lr=lr_stage1) for model in client_models]
#opt = [optim.SGD(model.parameters(), lr=lr_stage1, weight_decay=weight_decay, momentum=momentum) for model in client_models]

### __imple de FedAvg__

In [15]:
# Run TCT-Stage1 (i.e., FedAvg)
for r in range(num_rounds_stage1):
    # load global weights
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

    # client update
    loss = 0
    for i in range(num_clients):
        local_loss = client_update(client_models[i], opt[i], train_loader[i], epoch=epochs_stage1)
        print("client i : ", i, " loss : ", local_loss)
        loss += local_loss

    # average params across neighbors
    average_models(global_model, client_models)

    # evaluate
    test_losses, accuracies = evaluate_many_models(client_models, test_loader)
    torch.save(client_models[0].state_dict(), 'data/ckpt_stage1/model_tct_stage1.pth')

    print('%d-th round: average train loss %0.3g | average test loss %0.3g | average test acc: %0.3f' % (
        #r, loss.float() / num_clients, test_losses.float().mean(), accuracies.float().mean()))
        r, loss / num_clients, test_losses.mean(), accuracies.mean()))

client i :  0  loss :  1.7062326669692993
client i :  1  loss :  1.8570561408996582
client i :  2  loss :  2.199442148208618
client i :  3  loss :  2.2541074752807617
client i :  4  loss :  2.305490255355835
0-th round: average train loss 2.06 | average test loss -1.11 | average test acc: 0.227
client i :  0  loss :  2.3289525508880615
client i :  1  loss :  1.8467363119125366
client i :  2  loss :  2.159747838973999
client i :  3  loss :  2.6263694763183594
client i :  4  loss :  1.5973689556121826
1-th round: average train loss 2.11 | average test loss -1.1 | average test acc: 0.223
client i :  0  loss :  1.6142537593841553
client i :  1  loss :  1.8455986976623535
client i :  2  loss :  1.6730669736862183
client i :  3  loss :  1.677009105682373
client i :  4  loss :  1.5675108432769775
2-th round: average train loss 1.68 | average test loss -1.53 | average test acc: 0.232
client i :  0  loss :  1.1045231819152832
client i :  1  loss :  1.1039862632751465
client i :  2  loss :  1.60

KeyboardInterrupt: 

In [17]:
torch.save(global_model.state_dict(), 'model_tct_stage1_ResNet18_Cifar10_0.1.pth')

## __Stage 2__

### __hyperparamètres__

In [ ]:
num_rounds_stage2 = args["rounds_stage2"]
batch_size = args["num_samples_per_client"]

### __modèle eNTK__

In [ ]:
# Init and load model ckpt
global_model = Net_eNTK() #supprimer .cuda()
global_model = ResNet18() #supprimer .cuda()

global_model.load_state_dict(torch.load(r'data/ckpt_stage1/model_tct_stage1.pth'))
#global_model.fc2 = nn.Linear(128, 1) #supprimer .cuda() #récupérer une unique sortie ici #supprimer le dernier layer pour le remplacer (passer de 128->10 à 128->1)

global_model.load_state_dict(torch.load(r'C:\Users\dmgtr\chocoEA\ResNet18_stage1.pth'))
global_model.fc = nn.Linear(512, 1) #supprimer .cuda() #récupérer une unique sortie ici #supprimer le dernier layer pour le remplacer (passer de 128->10 à 128->1)

print('load model')

In [ ]:
model18 = ResNet18()
model14 = ResNet14()

model18.load_state_dict(torch.load(r'C:\Users\dmgtr\chocoEA\ResNet18_stage1.pth'))

In [ ]:
# Transfert des poids des couches convolutives et des BatchNorm2d

# Conv1
model14.conv1.weight.data = model18.conv1.weight.data.clone()
model14.bn1.weight.data = model18.bn1.weight.data.clone()
model14.bn1.bias.data = model18.bn1.bias.data.clone()
model14.bn1.running_mean = model18.bn1.running_mean.clone()
model14.bn1.running_var = model18.bn1.running_var.clone()

# Layer1
for i in range(2):
    model14.layer1[i].conv1.weight.data = model18.layer1[i].conv1.weight.data.clone()
    model14.layer1[i].bn1.weight.data = model18.layer1[i].bn1.weight.data.clone()
    model14.layer1[i].bn1.bias.data = model18.layer1[i].bn1.bias.data.clone()
    model14.layer1[i].bn1.running_mean = model18.layer1[i].bn1.running_mean.clone()
    model14.layer1[i].bn1.running_var = model18.layer1[i].bn1.running_var.clone()

    model14.layer1[i].conv2.weight.data = model18.layer1[i].conv2.weight.data.clone()
    model14.layer1[i].bn2.weight.data = model18.layer1[i].bn2.weight.data.clone()
    model14.layer1[i].bn2.bias.data = model18.layer1[i].bn2.bias.data.clone()
    model14.layer1[i].bn2.running_mean = model18.layer1[i].bn2.running_mean.clone()
    model14.layer1[i].bn2.running_var = model18.layer1[i].bn2.running_var.clone()

for i in range(2):
    model14.layer2[i].conv1.weight.data = model18.layer2[i].conv1.weight.data.clone()
    model14.layer2[i].bn1.weight.data = model18.layer2[i].bn1.weight.data.clone()
    model14.layer2[i].bn1.bias.data = model18.layer2[i].bn1.bias.data.clone()
    model14.layer2[i].bn1.running_mean = model18.layer2[i].bn1.running_mean.clone()
    model14.layer2[i].bn1.running_var = model18.layer2[i].bn1.running_var.clone()

    model14.layer2[i].conv2.weight.data = model18.layer2[i].conv2.weight.data.clone()
    model14.layer2[i].bn2.weight.data = model18.layer2[i].bn2.weight.data.clone()
    model14.layer2[i].bn2.bias.data = model18.layer2[i].bn2.bias.data.clone()
    model14.layer2[i].bn2.running_mean = model18.layer2[i].bn2.running_mean.clone()
    model14.layer2[i].bn2.running_var = model18.layer2[i].bn2.running_var.clone()

for i in range(2):
    model14.layer3[i].conv1.weight.data = model18.layer3[i].conv1.weight.data.clone()
    model14.layer3[i].bn1.weight.data = model18.layer3[i].bn1.weight.data.clone()
    model14.layer3[i].bn1.bias.data = model18.layer3[i].bn1.bias.data.clone()
    model14.layer3[i].bn1.running_mean = model18.layer3[i].bn1.running_mean.clone()
    model14.layer3[i].bn1.running_var = model18.layer3[i].bn1.running_var.clone()

    model14.layer3[i].conv2.weight.data = model18.layer3[i].conv2.weight.data.clone()
    model14.layer3[i].bn2.weight.data = model18.layer3[i].bn2.weight.data.clone()
    model14.layer3[i].bn2.bias.data = model18.layer3[i].bn2.bias.data.clone()
    model14.layer3[i].bn2.running_mean = model18.layer3[i].bn2.running_mean.clone()
    model14.layer3[i].bn2.running_var = model18.layer3[i].bn2.running_var.clone()


# Layer2 et Layer3
# Répétez un processus similaire pour layer2 et layer3 de model14 et model18

# Notez que pour les couches avec downsample, vous devrez également copier ces poids si elles existent


In [ ]:
model14.fc = nn.Linear(256, 1) #supprimer .cuda() #récupérer une unique sortie ici #supprimer le dernier layer pour le remplacer (passer de 512->10 à 512->1)
global_model = model14

### __Compute eNTK__

In [ ]:
# Train
grad_all = []
target_all = []
target_onehot_all = []
for i in range(num_clients):
    # Modify the batch size in the training data loader to match the model's batch size
    train_loader[i] = torch.utils.data.DataLoader(traindata_split[i], batch_size=batch_size_stage1, shuffle=True)
    
    grad_i, target_onehot_i, target_i = client_compute_eNTK(global_model, train_loader[i])
    grad_all.append(copy.deepcopy(grad_i).cpu())
    target_all.append(copy.deepcopy(target_i).cpu())
    target_onehot_all.append(copy.deepcopy(target_onehot_i).cpu())
    del grad_i
    del target_onehot_i
    del target_i
    torch.cuda.empty_cache()

In [ ]:
# Test
grad_eval, target_eval_onehot, target_eval  = client_compute_eNTK(global_model, test_loader)

### __run stage 2__

In [ ]:
# Init linear models
theta_global = torch.zeros(100000, 10) #supprimer .cuda()

client_thetas = [torch.zeros_like(theta_global) for _ in range(num_clients)] #supprimer .cuda()
client_hi_s = [torch.zeros_like(theta_global) for _ in range(num_clients)] #supprimer .cuda()

__prendre pour theta initial la valeur des paramètres du modèle -> échec__

In [ ]:
#seed = args['seed']
#subsample_size = 100000
#torch.manual_seed(seed)
#torch.cuda.manual_seed(seed)

#n_params = sum(p.numel() for p in list(global_model.parameters()))
#random_index = torch.randperm(n_params)[:subsample_size]
#theta_global = torch.cat([param.flatten() for param in list(model.parameters())])[random_index]

#theta_global = theta_global.view(-1, 1).repeat(1, 10)
#client_thetas = [theta_global for _ in range(num_clients)] #supprimer .cuda()

In [ ]:
# Run TCT-Stage2
for round_idx in range(num_rounds_stage2):
    theta_list = []
    for i in range(num_clients):
        theta_hat_update, h_i_client_update = scaffold_update(grad_all[i],
                                                              target_all[i],
                                                              client_thetas[i],
                                                              client_hi_s[i],
                                                              theta_global,
                                                              M=args["local_steps_stage2"],
                                                              lr_local=args["local_lr_stage2"])
        client_hi_s[i] = h_i_client_update * 1.0
        client_thetas[i] = theta_hat_update * 1.0
        theta_list.append(theta_hat_update)

    # averaging
    theta_global = torch.zeros_like(theta_list[0]) #supprimer .cuda()
    for theta_idx in range(num_clients):
        theta_global += (1.0 / num_clients) * theta_list[theta_idx]

    # eval on train
    logits_class_train = torch.cat(grad_all) @ theta_global #supprimer .cuda()
    _, targets_pred_train = logits_class_train.max(1)
    train_acc = targets_pred_train.eq(torch.cat(target_all)).sum() / (1.0 * logits_class_train.shape[0]) #supprimer .cuda()
    # eval on test
    logits_class_test = grad_eval @ theta_global
    _, targets_pred_test = logits_class_test.max(1)
    test_acc = targets_pred_test.eq(target_eval).sum() / (1.0 * logits_class_test.shape[0]) #supprimer .cuda()
    print('Round %d: train accuracy=%0.5g test accuracy=%0.5g' % (round_idx, train_acc.item(), test_acc.item()))